In [1]:
import os


fslpath = "/home/amatys/fsl"
os.environ["FSLDIR"] = fslpath
os.environ["PATH"] += os.pathsep + os.path.join(fslpath, 'bin')
!. ${FSLDIR}/etc/fslconf/fsl.sh

!flirt -version

FLIRT version 6.0


In [5]:
def register_fsl(
        moving_image,
        fixed_image,
        out_image=None,
        mat_path=None,
        return_command=False,
        transformation="rigid",
        interp="nearestneighbour",
        overwrite=False,
):
    dof = 9  # 9 is affine
    interp = "nearestneighbour" if interp in ['nearestneighbour', 'nearest'] \
        else "trilinear"

    atlas_name = os.path.split(fixed_image)[1].split(".")[0]
    mv_img_p, mv_img_n = os.path.split(moving_image)

    out_dir = os.path.join(mv_img_p, atlas_name)
    #veri_folder(out_dir)

    out_image = (
        os.path.join(out_dir, mv_img_n) if out_image is None else out_image
    )

    mat_path = (
        os.path.join(out_dir, mv_img_n.replace(".nii.gz", ".mat"))
        if mat_path is None
        else mat_path
    )

    command = (
        'flirt -in "{}" '
        '-ref "{}" '
        '-out "{}" '
        '-omat "{}" '
        "-bins 256 "
        "-cost corratio "
        "-searchrx -90 90 "
        "-searchry -90 90 "
        "-searchrz -90 90 "
        "-dof {}  "
        "-interp {}".format(
            moving_image, fixed_image, out_image, mat_path, dof, interp
        )
    )
    if return_command:
        return command

    if os.path.exists(out_image) and not overwrite and os.path.getsize(
            out_image) > 0:
        print("\n  The output file already exists (skipping).")
        return

    os.environ["OMP_NUM_THREADS"] = "10"
    os.system(command)

# Conform

In [1]:
import nibabel as nib

In [ ]:
nii_br = nib.load("")
nii = nib.load('')

In [3]:
import nibabel.processing as nipro

nii_proc = nipro.conform(nii , out_shape=nii_br.header.get_data_shape(), voxel_size=nii_br.header.get_zooms())

In [4]:
print(nii_proc.header.get_data_shape())
print(nii_proc.header.get_zooms())

(240, 240, 155)
(1.0, 1.0, 1.0)


In [ ]:
import os
import subprocess
#import nibabel as nib

def run_flirt_with_conform(input_dir):
    input_files = []
    output_dir = os.path.join(input_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    # Find input files
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(("0000.nii.gz", "0001.nii.gz", "0002.nii.gz", "0003.nii.gz", "seg.nii.gz")):
                input_files.append(os.path.join(root, file))

    # Find the '0001.nii.gz' file and apply the conform
    conformed_reference_file = None
    for input_file in input_files:
        if input_file.endswith('0001.nii.gz'):
            base_name = os.path.splitext(os.path.basename(input_file))[0]
            conformed_reference_file = os.path.join(output_dir, f"{base_name}_new.nii.gz")
            img = nib.load(input_file)
            conformed_img = nib.processing.conform(img, out_shape=(240, 240, 155), voxel_size=(1.0, 1.0, 1.0))
            nib.save(conformed_img, conformed_reference_file)
            print(f"Processed: {input_file}")
            break

    if conformed_reference_file is None:
        print("No '0001.nii.gz' file found for reference.")
        return

    # Run FLIRT using the conformed reference file
    for input_file in input_files:
        if input_file == conformed_reference_file:
            continue  # Skip processing the conformed reference file

        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_file = os.path.join(output_dir, f"{base_name}_new.nii.gz")
        mat_file = os.path.join(output_dir, f"{base_name}_new.mat")

        if input_file.endswith('seg.nii.gz'):
            interp = "nearestneighbour"
        else:
            interp = "trilinear"

        if not input_file.endswith('0001.nii.gz'):
            cmd = f"flirt -in {input_file} -ref {conformed_reference_file} -out {output_file} -omat {mat_file} -dof 12 -cost mutualinfo -interp {interp}"
            subprocess.run(cmd, shell=True, check=True)

        #cmd = f"flirt -in {input_file} -ref {conformed_reference_file} -out {output_file} -omat {mat_file} -dof 12 -cost mutualinfo -interp {interp}"
        #subprocess.run(cmd, shell=True, check=True)
        print(f"Processed: {input_file}")

    print("All files processed successfully.")

# Usage example
input_directory = ""
run_flirt_with_conform(input_directory)

## OLD

In [ ]:
import os
import subprocess
import nibabel as nib

def run_flirt_with_conform(input_dir):
    input_files = []
    output_dir = os.path.join(input_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    # Find input files
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(("0000.nii.gz", "0001.nii.gz", "0002.nii.gz", "0003.nii.gz", "seg.nii.gz")):
                input_files.append(os.path.join(root, file))

    # Run flirt and apply conform for each input file
    for input_file in input_files:
        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_file = os.path.join(output_dir, f"{base_name}_new.nii.gz")
        mat_file = os.path.join(output_dir, f"{base_name}_new.mat")

        if input_file.endswith('0001.nii.gz'):
            # Apply conform directly without running FLIRT
            img = nib.load(input_file)
            conformed_img = nib.processing.conform(img, out_shape=(240, 240, 155), voxel_size=(1.0, 1.0, 1.0))
            conformed_file = os.path.join(output_dir, f"{base_name}_new.nii.gz")
            nib.save(conformed_img, conformed_file)
            print(f"Processed: {input_file}")
        else:
            ref_file = input_file.replace('0000.nii.gz', '0001.nii.gz')
            if ref_file == input_file:
                ref_file = ref_file.replace('0001.nii.gz', '0000.nii.gz')

            cmd = f"flirt -in {input_file} -ref {ref_file} -out {output_file} -omat {mat_file} -dof 12 -cost mutualinfo -interp trilinear"
            subprocess.run(cmd, shell=True, check=True)

            # Apply conform
            img = nib.load(output_file)
            conformed_img = nib.processing.conform(img, out_shape=(240, 240, 155), voxel_size=(1.0, 1.0, 1.0))
            conformed_file = os.path.join(output_dir, f"{base_name}_new.nii.gz")
            nib.save(conformed_img, conformed_file)

            print(f"Processed: {input_file}")

    print("All files processed successfully.")

# Usage example
input_directory = ""
run_flirt_with_conform(input_directory)